In [24]:
!pip install requests python-docx

In [25]:
# Configurações do Azure Translator
subscription_key = 'SEU_AZURE_TRANSLATOR_SUBSCRIPTION_KEY'
endpoint = 'https://api.cognitive.microsofttranslator.com/'
region = 'SEU_REGION'  # ex: brazilsouth

In [ ]:
import requests
import json
from docx import Document
from docx.shared import Pt
from IPython.display import display
import ipywidgets as widgets
from io import BytesIO
import base64

# Função para traduzir texto
def translate_text(text, from_lang, to_lang):
    path = '/translate'
    constructed_url = endpoint + path

    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Ocp-Apim-Subscription-Region': region,
        'Content-type': 'application/json',
    }

    body = [{
        'text': text
    }]

    params = {
        'api-version': '3.0',
        'from': from_lang,
        'to': [to_lang]
    }

    response = requests.post(constructed_url, headers=headers, params=params, json=body)
    response.raise_for_status()

    translation = response.json()
    return translation[0]['translations'][0]['text']

# Função para carregar e traduzir um documento docx
def translate_document(file_path, from_lang, to_lang):
    doc = Document(file_path)
    translated_doc = Document()

    for paragraph in doc.paragraphs:
        if paragraph.text:
            translated_text = translate_text(paragraph.text, from_lang, to_lang)
            new_paragraph = translated_doc.add_paragraph(translated_text)
            new_paragraph.style.font.size = Pt(12)

    # Salva o documento traduzido em memória para permitir download
    buffer = BytesIO()
    translated_doc.save(buffer)
    buffer.seek(0)

    return buffer

# Função para criar um link de download do arquivo em memória
def create_download_link(buffer, filename):
    # Converte o conteúdo do arquivo em base64
    b64 = base64.b64encode(buffer.getvalue()).decode()
    href = f'<a download="{filename}" href="data:application/vnd.openxmlformats-officedocument.wordprocessingml.document;base64,{b64}">Clique aqui para baixar o arquivo traduzido</a>'
    return widgets.HTML(value=href)

# Widgets para a interface de upload de arquivo e seleção de idiomas
file_upload = widgets.FileUpload(description="Upload do Documento (.docx)", accept='.docx')
from_lang = widgets.Text(description="Idioma de Origem:", value="en")
to_lang = widgets.Text(description="Idioma de Destino:", value="es")
translate_button = widgets.Button(description="Traduzir")
output = widgets.Output()
alert = widgets.Label(value="")

# Função de alerta ao carregar o arquivo
def on_file_upload_change(change):
    if file_upload.value:
        alert.value = "Arquivo carregado com sucesso! Pronto para tradução."

# Função de tradução ao clicar no botão
def on_translate_button_clicked(b):
    with output:
        output.clear_output()
        if file_upload.value:
            # Extrai o arquivo do widget de upload
            file_info = next(iter(file_upload.value.values()))
            file_path = BytesIO(file_info['content'])

            # Traduz o documento
            translated_file = translate_document(file_path, from_lang.value, to_lang.value)

            # Cria o link de download direto em memória
            filename = f"traduzido_{file_info['metadata']['name']}"
            download_link = create_download_link(translated_file, filename)

            # Exibe o link de download
            display(download_link)
        else:
            print("Por favor, faça o upload de um documento .docx")

# Conecta a função ao botão de tradução e alerta de carregamento
file_upload.observe(on_file_upload_change, names='value')
translate_button.on_click(on_translate_button_clicked)

# Exibe widgets e output
display(file_upload, alert, from_lang, to_lang, translate_button, output)
